In [3]:
import  pandas as pd
from sklearn import metrics
import numpy as np
# from sklearn.metrics import r2_score, roc_curve, ndcg_score

model_name = "ePtsRep"
vary_bright = 1
step2_bright_degree = 0.6
# seed = 0
# classfy_scor = 1

step1_acc, step1_auc, step2_acc, step2_auc = 0, 0, 0, 0
step1_classfy_1, step1_classfy_9, step1_classfy_11 = 0, 0, 0
step2_classfy_1, step2_classfy_9, step2_classfy_11 = 0, 0, 0
step1_acc_list, step1_auc_list, step2_acc_list, step2_auc_list = [], [], [], []
step1_classfy_1_list, step1_classfy_9_list, step1_classfy_11_list = [], [], []
step2_classfy_1_list, step2_classfy_9_list, step2_classfy_11_list = [], [], []


def ndcg(y_pred, y_true):
    y_true_normalized = (y_true - y_true.mean()) / y_true.std()
    return metrics.ndcg_score(y_true_normalized.reshape(1, -1), y_pred.reshape(1, -1))

def classify(a, b, thld):
    pred = a
    tgt = b

    tp = fp = tn = fn = 1e-9
    pred_bin = []
    real_bin = []
    for i in range(len(tgt)):
        pred_i = pred[i]
        tgt_i = tgt[i]
        if pred_i > thld and tgt_i > thld:
            tp += 1
        if pred_i < thld and tgt_i > thld:
            fn += 1
        if pred_i > thld and tgt_i < thld:
            fp += 1
        if pred_i < thld and tgt_i < thld:
            tn += 1

        if pred_i >= thld:
            pred_bin.append(1)
        else:
            pred_bin.append(0)
        if tgt_i >= thld:
            real_bin.append(1)
        else:
            real_bin.append(0)
    # print(tp, tn ,fp ,fn)
    acc = (tp + tn) / (tp + tn + fp + fn)
    acc_t = tp / (tp + fp)
    acc_f = tn / (tn + fn)
    recall_t = tp / (tp + fn)
    recall_f = tn / (tn + fp)
    fpr, tpr, _ = metrics.roc_curve(real_bin, pred_bin)
    auc = metrics.auc(fpr, tpr)
    return acc, auc

def get_classfy_num(dataframe, index):
    sort_num = 100
    dataframe.sort_values(by = index, ascending=False, inplace = True)
    dataframe = dataframe.reset_index(drop = True)
    df_bright_sort = dataframe[:sort_num]
    # print(list(df_bright_sort["quantitative_function"]))
    num_1 = 0
    num_9 = 0
    num_11 = 0
    for i in list(df_bright_sort["quantitative_function"]):
        if i > 1:
            num_1 += 1
        if i > 0.9:
            num_9 += 1
        if i > 1.1:
            num_11 += 1
    # print("calssfy 1: " ,num_1, num_1 / sort_num)
    # print("calssfy 0.9: " ,num_9, num_9 / sort_num)
    # print("calssfy 1.1: " ,num_11, num_11 / sort_num)
    return num_1, num_9, num_11
    

In [4]:
for seed in range(5):
    if vary_bright:
        input_path = f"/share/jake/hot_spot/data/test/method4/step2/result_csv/{model_name}_calssfy_degree{step2_bright_degree}_result_step2_vary_bright_{seed}.csv"
    else:
        input_path = f"/share/jake/hot_spot/data/test/method4/step2/result_csv/{model_name}_calssfy_degree{step2_bright_degree}_result_step2_bright_{seed}.csv"
    dataframe = pd.read_csv(input_path)
    df_bright = dataframe[dataframe["quantitative_function"] > 1]
    bright_num = len(list(df_bright["name"]))
    real_qfunc = np.array(list(dataframe["quantitative_function"]))
    predict_qfunc_1 = np.array(list(dataframe["predict_qfunc"]))
    predict_qfunc_2 = np.array(list(dataframe["predict_qfunc_step2"]))
    acc_1, auc_1 = classify(predict_qfunc_1, real_qfunc, 1)
    # ndcg_1 = ndcg(predict_qfunc_1, real_qfunc)
    step1_acc_list.append(acc_1)
    step1_acc += acc_1
    step1_auc_list.append(auc_1)
    step1_auc += auc_1
    acc_2, auc_2 = classify(predict_qfunc_2, real_qfunc, 1)
    # ndcg_2 = ndcg(predict_qfunc_2, real_qfunc)
    step2_acc_list.append(acc_2)
    step2_acc += acc_2
    step2_auc_list.append(auc_2)
    step2_auc += auc_2
    step1_classfy_1_seed, step1_calssfy_9_seed, step1_calssfy_11_seed = get_classfy_num(dataframe, "predict_qfunc")
    step1_classfy_1_list.append(step1_classfy_1_seed)
    step1_classfy_1 += step1_classfy_1_seed
    step1_classfy_9_list.append(step1_calssfy_9_seed)
    step1_classfy_9 += step1_calssfy_9_seed
    step1_classfy_11_list.append(step1_calssfy_11_seed)
    step1_classfy_11 += step1_calssfy_11_seed
    step2_classfy_1_seed, step2_calssfy_9_seed, step2_calssfy_11_seed = get_classfy_num(dataframe, "predict_qfunc_step2")
    step2_classfy_1_list.append(step2_classfy_1_seed)
    step2_classfy_1 += step2_classfy_1_seed
    step2_classfy_9_list.append(step2_calssfy_9_seed)
    step2_classfy_9 += step2_calssfy_9_seed
    step2_classfy_11_list.append(step2_calssfy_11_seed)
    step2_classfy_11 += step2_calssfy_11_seed
print(model_name)
# print("step1", step1_acc_list, step1_auc_list)
print("step1 auc: ", step1_auc / 5)
# print("step2", step2_acc_list, step2_auc_list)
print("step2 auc: ", step2_auc / 5)
# print("step1", step1_classfy_1_list, step1_classfy_9_list, step1_classfy_11_list)
# print("step2", step2_classfy_1_list, step2_classfy_9_list, step2_classfy_11_list)
print("step1 classfy 1.0: ", step1_classfy_1 / 5, "step1 classfy 0.9: ", step1_classfy_9 / 5, "step1 classfy 1.1: ", step1_classfy_11 / 5)
print("step2 classfy 1.0: ", step2_classfy_1 / 5, "step2 classfy 0.9: ", step2_classfy_9 / 5, "step2 classfy 1.1: ", step2_classfy_11 / 5)

step1 [0.12489130434798915, 0.12934782608711765, 0.12228260869581638, 0.12782608695668352, 0.12923913043494378] [0.5, 0.5, 0.5, 0.5, 0.5]
step1 auc:  0.5
step2 [0.8108695652172561, 0.7949999999998717, 0.810108695652039, 0.7926086956520466, 0.8038043478259548] [0.5670063095373608, 0.5509992761149405, 0.5448765049879601, 0.5472782333272291, 0.5514377885999685]
step2 auc:  0.5523196225134918
step1 [12, 24, 11, 18, 15] [38, 68, 30, 58, 59] [0, 2, 0, 0, 0]
step2 [21, 21, 19, 21, 24] [83, 69, 80, 68, 77] [1, 0, 0, 1, 0]
step1 classfy 1.0:  16.0 step1 classfy 0.9:  50.6 step1 classfy 1.1:  0.4
step2 classfy 1.0:  21.2 step2 classfy 0.9:  75.4 step2 classfy 1.1:  0.4


In [64]:
sort_num = 100
dataframe.sort_values(by = "predict_qfunc", ascending=False, inplace = True)
dataframe = dataframe.reset_index(drop = True)
df_bright_sort = dataframe[:sort_num]
# print(list(df_bright_sort["quantitative_function"]))
num_1 = 0
num_9 = 0
num_11 = 0
for i in list(df_bright_sort["quantitative_function"]):
    if i > 1:
        num_1 += 1
    if i > 0.9:
        num_9 += 1
    if i > 1.1:
        num_11 += 1
print("calssfy 1: " ,num_1, num_1 / sort_num)
print("calssfy 0.9: " ,num_9, num_9 / sort_num)
print("calssfy 1.1: " ,num_11, num_11 / sort_num)

calssfy 1:  17 0.17
calssfy 0.9:  63 0.63
calssfy 1.1:  0 0.0


In [65]:
dataframe.sort_values(by = "predict_qfunc_step2", ascending=False, inplace = True)
dataframe = dataframe.reset_index(drop = True)
df_bright_sort = dataframe[:sort_num]
# print(list(df_bright_sort["quantitative_function"]))
# num = 0
# for i in list(df_bright_sort["quantitative_function"]):
#     if i > classfy_scor:
#         num += 1
# print(num, num / sort_num)
num_1 = 0
num_9 = 0
num_11 = 0
for i in list(df_bright_sort["quantitative_function"]):
    if i > 1:
        num_1 += 1
    if i > 0.9:
        num_9 += 1
    if i > 1.1:
        num_11 += 1
print("calssfy 1: " ,num_1, num_1 / sort_num)
print("calssfy 0.9: " ,num_9, num_9 / sort_num)
print("calssfy 1.1: " ,num_11, num_11 / sort_num)

calssfy 1:  27 0.27
calssfy 0.9:  83 0.83
calssfy 1.1:  0 0.0
